In [33]:
import pandas as pd

df = pd.read_csv("hf://datasets/bitext/Bitext-telco-llm-chatbot-training-dataset/bitext-telco-llm-chatbot-training-dataset.csv")

In [34]:
df.head()


,instruction,intent,category,tags,response
0,there are charges on my phone bill that i do n...,dispute_invoice,BILLING,BCELQZ,If you have noticed discrepancies in your bill...
1,"my internet bill is incorrect, can you help me...",dispute_invoice,BILLING,BCILZ,If there is an issue with your bill and you wo...
2,my invoice is not corect challenge it,dispute_invoice,BILLING,BCELQZ,If you find any discrepancies in your invoice ...
3,"I don't recognize some charges on my bill, can...",dispute_invoice,BILLING,BCIL,If you have found charges on your bill that yo...
4,i have to dispute a fucking bill i need help,dispute_invoice,BILLING,BCQW,If you believe there is an error on your bill ...


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26000 entries, 0 to 25999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  26000 non-null  object
 1   intent       26000 non-null  object
 2   category     26000 non-null  object
 3   tags         26000 non-null  object
 4   response     26000 non-null  object
dtypes: object(5)
memory usage: 1015.8+ KB


In [38]:

print("category :", df["category"].unique())

print("intent :", df['intent'].unique())

category : ['BILLING' 'COMPLAINTS' 'CONSUMPTION' 'CONTACT' 'PAYMENT' 'SERVICES'
 'SUBSCRIPTION']
intent : ['dispute_invoice' 'invoices' 'get_compensation'
 'report_poor_signal_coverage' 'report_problem'
 'check_excess_data_charges' 'check_usage' 'set_usage_limits'
 'customer_service' 'human_agent' 'check_mobile_payments'
 'payment_methods' 'pay' 'schedule_payments'
 'activate_call_management_services' 'activate_phone' 'activate_roaming'
 'check_signal_coverage' 'deactivate_call_management_services'
 'deactivate_phone' 'install_internet' 'cancel_plan' 'change_plan'
 'change_provider' 'check_cancellation_fee' 'sign_up_for_plan']


In [39]:
df1 = pd.read_csv("synthetic_query_dataset.csv")
df1.head()

,instruction,intent,category,tags,response
0,Show me my latest invoice.,invoices,BILLING,"billing,invoices",We can help you with invoices. Please follow t...
1,I want to pay my bill.,pay,PAYMENT,"payment,pay",We can help you with pay. Please follow the in...
2,I want to speak to a human agent.,human_agent,CONTACT,"contact,human_agent",We can help you with human agent. Please follo...
3,What payment methods are available?,payment_methods,PAYMENT,"payment,payment_methods",We can help you with payment methods. Please f...
4,How much data have I used?,check_usage,CONSUMPTION,"consumption,check_usage",We can help you with check usage. Please follo...


In [40]:
df.drop(columns=['tags'], inplace=True)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26000 entries, 0 to 25999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  26000 non-null  object
 1   intent       26000 non-null  object
 2   category     26000 non-null  object
 3   response     26000 non-null  object
dtypes: object(4)
memory usage: 812.6+ KB


In [43]:
df1.drop(columns=['tags'], inplace=True)

In [44]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  3500 non-null   object
 1   intent       3500 non-null   object
 2   category     3500 non-null   object
 3   response     3500 non-null   object
dtypes: object(4)
memory usage: 109.5+ KB


In [47]:
from langchain_core.documents import Document
document = []

for _, row in df.iterrows():
    content = f"""
Category: {row['category']}
User Intent: {row['instruction']}
Agent Response: {row['response']}
Intent: {row.get('intent', '')}
"""
    document.append(
        Document(
            page_content=content.strip(),
            metadata={
                "category": row["category"],
                "priority": row.get("priority", "normal")
            }
        )
    )

for _, row in df1.iterrows():
    content = f"""
Category: {row['category']}
User Intent: {row['instruction']}
Agent Response: {row['response']}
Intent: {row.get('intent', '')}
"""
    document.append(
        Document(
            page_content=content.strip(),
            metadata={
                "category": row["category"],
                "priority": row.get("priority", "normal")
            }
        )
    )
    

In [48]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = splitter.split_documents(document)


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5"
)

# Test it immediately to ensure it works
vector = embeddings.embed_query("Hello world")
print(len(vector))  # Should print 384 for BGE-small